In [1]:
import pandas

import pyspark

In [2]:
%load_ext sparkmagic.magics

In [3]:
%manage_spark

MagicsControllerWidget(children=(Tab(children=(ManageSessionWidget(children=(HTML(value='<br/>'), HTML(value='…

In [13]:
%manage_spark?

Docstring:
Magic to manage Spark endpoints and sessions. First, add an endpoint via the 'Add Endpoint' tab.
Then, create a session. You'll be able to select the session created from the %%spark magic.
File:      /opt/conda/lib/python3.7/site-packages/sparkmagic/magics/remotesparkmagics.py


In [11]:
%%spark

import random
NUM_SAMPLES = 100000
def sample(p):
  x, y = random.random(), random.random()
  return 1 if x*x + y*y < 1 else 0

count = sc.parallelize(range(0, NUM_SAMPLES)).map(sample).reduce(lambda a, b: a + b)
print(f"Pi is roughly {4.0 * count / NUM_SAMPLES}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Pi is roughly 3.1384

In [12]:
%%spark -c sql

SHOW TABLES

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

""


## Livy

In [4]:
import json, pprint, requests, textwrap
host = 'http://localhost:8998'
data = {'kind': 'pyspark'}
headers = {'Content-Type': 'application/json'}
r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
r.json()


{'id': 0,
 'name': None,
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'starting',
 'kind': 'spark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['stdout: ', '\nstderr: ']}

In [5]:
session_url = host + r.headers['location']
r = requests.get(session_url, headers=headers)
r.json()

{'id': 0,
 'name': None,
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'idle',
 'kind': 'spark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['20/04/24 10:55:41 INFO Executor: Using REPL class URI: spark://d2b1e510f4b2:39915/classes',
  "20/04/24 10:55:41 INFO Utils: Successfully started service 'org.apache.spark.network.netty.NettyBlockTransferService' on port 42011.",
  '20/04/24 10:55:41 INFO NettyBlockTransferService: Server created on d2b1e510f4b2:42011',
  '20/04/24 10:55:41 INFO BlockManager: Using org.apache.spark.storage.RandomBlockReplicationPolicy for block replication policy',
  '20/04/24 10:55:41 INFO BlockManagerMaster: Registering BlockManager BlockManagerId(driver, d2b1e510f4b2, 42011, None)',
  '20/04/24 10:55:41 INFO BlockManagerMasterEndpoint: Registering block manager d2b1e510f4b2:42011 with 366.3 MB RAM, BlockManagerId(driver, d2b1e510f4b2, 42011, None)',
  '20/04/24 10:55:41 INFO BlockManagerMaster: Registered BlockManager Block

In [7]:
data = {
  'code': textwrap.dedent("""
    import random
    NUM_SAMPLES = 100000
    def sample(p):
      x, y = random.random(), random.random()
      return 1 if x*x + y*y < 1 else 0

    count = sc.parallelize(xrange(0, NUM_SAMPLES)).map(sample).reduce(lambda a, b: a + b)
    print "Pi is roughly %f" % (4.0 * count / NUM_SAMPLES)
    """)
}

r = requests.post(statements_url, data=json.dumps(data), headers=headers)
pprint.pprint(r.json())

NameError: name 'statements_url' is not defined

In [ ]:
session_url = 'http://localhost:8998/sessions/0'
requests.delete(session_url, headers=headers)